---
### NLP chapter 3. Transformer deep 하게 이해하기.

# 3.1 transformer architecture
- 인코더 : 입력토큰의 시퀀스를 은닉상태 또는 문맥(context) 라 부르는 임베딩 벡터의 시퀀스로 반환
- 디코더 :" 




<br><img src = "https://www.notion.so/image/https%3A%2F%2Fprod-files-secure.s3.us-west-2.amazonaws.com%2F914ff162-e9aa-4454-80d2-30b702551154%2F099316f3-3caa-4843-83c2-8c85ecd87382%2FUntitled.png?table=block&id=52e74767-4875-4af9-aab4-571a9ecd0bb1&spaceId=914ff162-e9aa-4454-80d2-30b702551154&width=1710&userId=d72aebc7-576c-4f0e-9c69-1d5d32da7874&cache=v2" width="500" height="300"/><br>


In [2]:
## Forrest Park's notion  ## Updated 2024.07.03
### Service  Text Coloring option
from Functions import Service as S
def blue(str):return S.colored_text(str,'blue')
def yellow(str):return S.colored_text(str,'yellow')
def red(str):return S.colored_text(str,'red')
def green(str):return S.colored_text(str,'green')
def imd(str):return S.imd(green(str))
## 자연어처리 패키지 설치 
def NLPInstalls():
    import subprocess,sys
    import warnings ; warnings.filterwarnings('ignore')
    # pip가 없으면 pip를 설치
    try:import pip
    except ImportError:
        print("Install pip for python3")
        subprocess.call(['sudo', 'apt-get', 'install', 'python3-pip'])
    
    # tweepy 없으면 tweepy 설치
    try:import tweepy        
    except ModuleNotFoundError:
        print("Install tweepy")
        subprocess.call([sys.executable, "-m", "pip", "install", 'tweepy==3.10.0'])
    finally:import tweepy 
    
    # konlpy 없으면 konlpy 설치
    try:import konlpy
    except ModuleNotFoundError: 
        print("Install konlpy")
        subprocess.call([sys.executable, "-m", "pip", "install", 'konlpy'])
    finally:import konlpy
    
    # eunjeon 없으면 eunjeon 설치
    try:import eunjeon
    except ModuleNotFoundError: 
        print("Install eunjeon : eunjeon")
        subprocess.call([sys.executable, "-m", "pip", "install", 'eunjeon'])
    finally:import konlpy
    
    # datasets 없으면 datasets를 설치
    try:import datasets
    except ModuleNotFoundError: 
        print("Install datasets : datasets")
        subprocess.call([sys.executable, "-m", "pip", "install", 'datasets'])
    finally:import datasets
    
    # pytorch 없으면 pytorch 설치
    try:import torch
    except ModuleNotFoundError: 
        print("Install torch : pytorch")
        subprocess.call([sys.executable, "-m", "pip", "install", 'pytorch'])
    finally:import torch
    
    # transformers 없으면 transformers 설치
    try:import transformers
    except ModuleNotFoundError: 
        print("Install transformer : transformers")
        subprocess.call([sys.executable, "-m", "pip", "install", 'transformers'])
    finally:import transformers
        
    # UMAP 없으면 UMAP 설치
    try:import umap
    except ModuleNotFoundError: 
        print("Install umap : umap")
        subprocess.call([sys.executable, "-m", "pip", "install", 'umap'])
    finally:import umap
        
    # UMAP 없으면 UMAP 설치
    try:from umap import UMAP
    except ImportError: 
        print("Install umap : umap-learn")
        subprocess.call([sys.executable, "-m", "pip", "install", 'umap-learn'])
    finally:import umap
NLPInstalls()
print(yellow(f"📌 - PROGRAM START \n\t"))
## GPU setting (in MacOS)
import torch
print(yellow(f"📌 - MPS 장치를 지원 Build 여부 : "),torch.backends.mps.is_built())
print(yellow(f"📌 - MPS 장치 사용가능 여부 : "),torch.backends.mps.is_available())
print(yellow(f"📌 - GPU 사용Start"))
# device = torch.device("mps")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#### Jupyter Basic Setting ####
import pandas as pd,numpy as np
import matplotlib.pyplot as plt,seaborn as sns  # 시각화
import warnings; warnings.filterwarnings('ignore')  # 경고 무시
import sys,os 
from pprint import pprint


📌 - PROGRAM START 
	
📌 - MPS 장치를 지원 Build 여부 :  True
📌 - MPS 장치 사용가능 여부 :  True
📌 - GPU 사용Start


In [4]:
#<img src = "" width="300" height="300"/><br>
def imd(image_address,width =500, height=300):
    print(f'<br><img src = "{image_address}" width="{width}" height="{height}"/><br>')
    
imd("https://www.notion.so/image/https%3A%2F%2Fprod-files-secure.s3.us-west-2.amazonaws.com%2F914ff162-e9aa-4454-80d2-30b702551154%2F099316f3-3caa-4843-83c2-8c85ecd87382%2FUntitled.png?table=block&id=52e74767-4875-4af9-aab4-571a9ecd0bb1&spaceId=914ff162-e9aa-4454-80d2-30b702551154&width=1710&userId=d72aebc7-576c-4f0e-9c69-1d5d32da7874&cache=v2")

<br><img src = "https://www.notion.so/image/https%3A%2F%2Fprod-files-secure.s3.us-west-2.amazonaws.com%2F914ff162-e9aa-4454-80d2-30b702551154%2F099316f3-3caa-4843-83c2-8c85ecd87382%2FUntitled.png?table=block&id=52e74767-4875-4af9-aab4-571a9ecd0bb1&spaceId=914ff162-e9aa-4454-80d2-30b702551154&width=1710&userId=d72aebc7-576c-4f0e-9c69-1d5d32da7874&cache=v2" width="500" height="300"/><br>


---
### Transformer 의 유형
- 인코더 유형 : 
    * 텍스트 시퀀스 입력을 풍부한 수치표현으로 변환 
    * 텍스트 분류나 개체명 인식 같은 작업에 잘맞음. 
    * Bert, RoBert
- 디코더 유형 :
    * 시작 텍스트가 주어지면 가장 가능성 있는 다음 단어를 반복해 예측하는 식으로 시퀀스 자동완성
    * GPT , causal attention, autoregressive attention
    * 한 토큰에 대해 계산한 표현은 오직 왼쪽 문맥에 따라 달라짐. 

- 인코더-디코더 유형: 
    * 한텍스트의 시퀀스를 다른 시퀀스로 매핑하는 복잡한 모델링에 사용 
    * 기계번역과 요약작업에 적합. 
    * BART, T5 모델 

---
### 3.2 인코더
: 인코더층은 임베딩 시퀀스를 받아 다음 두 층에 통과시킨다.      
: 임베딩이란 자연어처리에서 사람이 쓰는 자연어를 기계가 이해할 수 있도록 숫자형태인 vector로 바꾸는 과정 혹은 일련의 전체 과정을 의미
- multihead self attention layer
- fully connected feed forward layer (각각의 입력 임베딩에 적용되는 완전 연결 피드 포워드층 )
<br><img src = "https://www.notion.so/image/https%3A%2F%2Fprod-files-secure.s3.us-west-2.amazonaws.com%2F914ff162-e9aa-4454-80d2-30b702551154%2F09d456af-00b9-4ab3-8a84-7fd2c463d27e%2FUntitled.png?table=block&id=eec10382-9a45-4f26-ae3f-9787fa035e64&spaceId=914ff162-e9aa-4454-80d2-30b702551154&width=1710&userId=d72aebc7-576c-4f0e-9c69-1d5d32da7874&cache=v2" width="500" height="300"/><br>


----
### 3.2.1  self attention 
- 텍스 시퀀스에서 원소는 토큰 임베딩임. 각 토큰은 고정차원의 벡터에 매핑됨. 
- 각 토큰에 대해 고정된 임베딩을 사용하는 대신 전체 시퀀스를 사용해  각 임베딩의 가중평균을 계산하는것이 셀프 어텐션의 기본 개념 

- 토큰임베딩 시퀀스 x1,x2,...,xn -> 새로운 임베딩 시퀀스 x'1,x'2,...
- 계수 wji -> 어텐션 가중치  전체합은 1 

